In [36]:
import h5py
import numpy as np
import scipy as sp

import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import pathlib

from obspy.signal.filter import bandpass
import scipy

from scipy import signal

from scipy.signal import hilbert

from scipy.io import savemat



from scipy.signal import detrend

from scipy.fft import fft

import random


import tensorflow as tf
from tensorflow import keras

from tensorflow.keras import regularizers

In [3]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt


from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalAveragePooling2D
from keras.models import Model


from keras.optimizers import Adam


In [ ]:
#### Normalizaciones

In [28]:
def oneBit(value):
    
    if value<=0:
        return -1

    else:
        return 1

In [29]:
OB = np.vectorize(oneBit)   

In [31]:
## data.shape[0] canales
## data.shape[1] tiempos
## normalizar trazas por desviación estándar de cada traza
def stdTrace(data):
    dataCP = data.copy()
    for i in range(0,data.shape[0]):
        dataCP[i,:] = dataCP[i,:]/(dataCP[i,:]).std()
        
    return dataCP
    
    

In [33]:

def compM(data,coef=2):
    #dataCpy = data.copy()
    #for traza in dataCpy:
    #Max = np.abs(data).max()
    #data = np.sign(data)*np.log(1+(10**coef)*np.abs(data)/Max)
    data = np.sign(data)*np.log(1+(10**coef)*np.abs(data))
    Max = np.abs(data).max()
    data = data/Max
    #dataCpy = np.log(1+(10**coef)*np.abs(data)/Max)
    return data

In [34]:

def RMScomp(X):
    A = X.copy()
    for i,traza in enumerate(A):
        rmsA = np.std((np.array(traza)))#rmsValue()
        filtered_list = traza.copy()
        rmsA = 10          
        rmsP = np.std((np.array(filtered_list)))

        while np.abs(rmsA - rmsP) >=1e-2:
            
            rmsP=rmsA
            filtered_list = filtered_list[np.abs(filtered_list-np.mean(filtered_list)) < 3*rmsP]
            
            rmsA = np.std((np.array(filtered_list)))
            
        A[i] = A[i]/rmsP
    return A

In [27]:
##### Sismos empleados para entrenamiento y validación

In [ ]:
fl1 = ["/StorageCitlalli/DAS/DataCDMX/2023/111/SR_DS_2023-04-21_16-31-12_UTC.h5", (526 - 0.5,536),(0,1100),[19.363,-99.177],665]#[19.363,-99.177] #0 #665 metros a la fibra
fl2 = ["/StorageCitlalli/DAS/DataCDMX/2023/110/SR_DS_2023-04-20_21-01-12_UTC.h5", (193,206), (0,1100),[19.362,-99.173],235]# #1 #235 metros a la fibra 
fl3 = ["/StorageCitlalli/DAS/DataCDMX/2023/101/SR_DS_2023-04-11_03-57-09_UTC.h5", (304.5,311),(500,1500),[19.428,-99.178],3350]#2 ## 3350 metros a la fibra 
fl4 = ["/StorageCitlalli/DAS/DataCDMX/2023/89/SR_DS_2023-03-30_00-44-42_UTC.h5", (368-0.5,374),(0,1000),[19.366,-99.183],1370]#3 ## 1370 metros hacia la fibra
fl5 = ["/StorageCitlalli/DAS/DataCDMX/2023/88/SR_DS_2023-03-29_21-24-42_UTC.h5", (131-0.4,137),(0,1000),[19.361,-99.177],580]#4 ## 580 metros a la fibra 
fl6 = ["/StorageCitlalli/DAS/DataCDMX/2023/88/SR_DS_2023-03-29_18-14-42_UTC.h5", (171-0.5,177),(0,1000),[19.358,-99.175],240]#5 ## 240 metros a la fibra 
fl7 = ["/StorageCitlalli/DAS/DataCDMX/2023/88/SR_DS_2023-03-29_13-24-42_UTC.h5", (584,590),(0,1000),[19.364082397377704,-99.183099866825570],1300]#6
fl10 = ["/StorageCitlalli/DAS/DataCDMX/2023/74/SR_DS_2023-03-15_10-12-13_UTC.h5", (555,570),(1000,2000),[19.358199637063734,-99.084083080089070],293]#7 ## Verificar tiempo

fl11 = ["/StorageCitlalli/DAS/DataCDMX/2023/74/SR_DS_2023-03-15_09-42-13_UTC.h5", (382,400),(1000,2000),[19.353782577578595,-99.086009099302600], 237]#8 # Verificar tiempo. Cambiar tiempo a 380 segundos y mover ventana en espacio a 1200-2200 canales 

 
fl12 = ["/StorageCitlalli/DAS/DataCDMX/2023/65/SR_DS_2023-03-06_22-43-54_UTC.h5", (412-0.5,418),(0,1000),[19.364,-99.19],1860]#9 ### 1860 metros a la fibra 


fl13 = ["/StorageCitlalli/DAS/DataCDMX/2023/57/SR_DS_2023-02-26_01-02-57_UTC.h5", (314-0.25,320),(0,1000),[19.359,-99.182],869]#10 #  869metros a la fibra
fl14 = ["/StorageCitlalli/DAS/DataCDMX/2023/54/SR_DS_2023-02-23_02-28-41_UTC.h5", (376,390),(1000,2000), [19.379588860184597,-99.106247307175750], 347 ]#11 ## 347 metros a la fibra
fl73 = ["/StorageCitlalli/DAS/DataCDMX/2023/53/SR_DS_2023-02-22_00-18-41_UTC.h5",(150+0.25,156),(0,1000),[19.364621960336606,-99.184510181015540],1460 ]#12 ## 1460 metros a la fibra 
fl16_0 = ["/StorageCitlalli/DAS/DataCDMX/2023/52/SR_DS_2023-02-21_22-48-41_UTC.h5", (31,40), (1150,2000),[19.386392234002237,-99.092788675449170], 1940]#13 ## 1940 metros a la fibra 
fl16 = ["/StorageCitlalli/DAS/DataCDMX/2023/52/SR_DS_2023-02-21_22-48-41_UTC.h5", (239,256), (1150,2000),[19.381094915634876,-99.093042749728800],1720]#14 ### 1720 metros a laa fibra
fl17 = ["/StorageCitlalli/DAS/DataCDMX/2023/52/SR_DS_2023-02-21_22-48-41_UTC.h5", (558,576),(1150,2000),[19.383509771578456,-99.092875480098800],1820]#15 ## 1820 metros a la fibra
fl19 = ["/StorageCitlalli/DAS/DataCDMX/2023/39/SR_DS_2023-02-08_16-09-38_UTC.h5", (397,403),(100,1000),[19.362160961821193,-99.176219015230200],554 ]#16


fl72 = ["/StorageCitlalli/DAS/DataCDMX/2023/122/SR_DS_2023-05-02_02-32-34_UTC.h5",(172,178),(100,1000),[19.366,-99.176],710] ## 17  710 metros a la fibra
fl71 = ["/StorageCitlalli/DAS/DataCDMX/2023/122/SR_DS_2023-05-02_04-42-34_UTC.h5",(493,499),(150,1000),[19.365,-99.157],1040] ###18  1040 metros
fl74 = ['/StorageCitlalli/DAS/DataCDMX/2023/53/SR_DS_2023-02-22_00-18-41_UTC.h5',(139.5,145.5),(50,1000),[19.361087826769563,-99.176073084584120],494]#19 ## 494 metros a la fibra

fl20 = ["/StorageCitlalli/DAS/DataCDMX/2023/38/SR_DS_2023-02-07_14-38-00_UTC.h5", (519,530),(0,1500),[19.359407876096930,-99.179649737792220],721]#20 ### 721 metros a la fibra
### Falta archivo de movimiento de hipocentros para:
fl21 = ["/StorageCitlalli/DAS/DataCDMX/2023/38/SR_DS_2023-02-07_14-28-00_UTC.h5", (529-0.75,539), (50,1000), [19.358919766724814,-99.175783156864600], 370]## 21 ##370 metros a la fibra 

fl22 = ["/StorageCitlalli/DAS/DataCDMX/2023/29/SR_DS_2023-01-29_18-46-18_UTC.h5", (174-0.15,184),(150,1000),[19.357,-99.155],1670]## 22 ##  1670 


fl23 = ["/StorageCitlalli/DAS/DataCDMX/2023/28/SR_DS_2023-01-28_15-20-55_UTC.h5", (296,303),(500,1500),[19.379,-99.156],330]## 23 ## 330 
fl24 = ["/StorageCitlalli/DAS/DataCDMX/2023/19/SR_DS_2023-01-19_08-51-17_UTC.h5", (584-1,594),(0,2000),[19.257342708319880,-99.205461421508820],7730]#24 ##7730 metros a la fibra 

fl25 = ["/StorageCitlalli/DAS/DataCDMX/2023/17/SR_DS_2023-01-17_09-05-04_UTC.h5", (229,235),(120,1000), [19.368982375562820,-99.175675109699140], 874]#25 ## 874 metros a la fibra 

fl26 = ["/StorageCitlalli/DAS/DataCDMX/2023/11/SR_DS_2023-01-11_15-24-39_UTC.h5", (100,106),(0,1000),[19.343,-99.181],47]#26 ## 47metros a la fibra 

fl27 = ["/StorageCitlalli/DAS/DataCDMX/2022/358/SR_DS_2022-12-24_20-11-44_UTC.h5", (558,562),(100,1000),[19.367230189174574,-99.185837488999350],1700]#27  ## 1700 metros a la fibra 

fl28 = ["/StorageCitlalli/DAS/DataCDMX/2022/351/SR_DS_2022-12-17_02-34-13_UTC.h5", (21-0.3,27),(200,1100),[19.369,-99.157],804]#28 # 804  ### __Revisar tiempos__ REVISADO

fl29 = ["/StorageCitlalli/DAS/DataCDMX/2022/349/SR_DS_2022-12-15_12-24-13_UTC.h5", (47+0.25,53),(100,1000),[19.368,-99.177],918]#29  #  918  ## __Verificar tiempo, parece estar mal__ REVISADO


fl30 = ["/StorageCitlalli/DAS/DataCDMX/2022/342/SR_DS_2022-12-08_07-28-45_UTC.h5", (502-0.25,508),(100,800),[19.372,-99.189],2230]#30 ## 2230

fl31 = ["/StorageCitlalli/DAS/DataCDMX/2022/342/SR_DS_2022-12-08_03-08-45_UTC.h5", (232,240),(400,1500) , [19.391527626181460, -99.162871642072570], 678]#31 ## 678 metros a la fibra    

fl32 = ["/StorageCitlalli/DAS/DataCDMX/2022/336/SR_DS_2022-12-02_20-41-22_UTC.h5", (35-0.3,43),(200,1000),[19.364,-99.178],807]#32 # 807


fl34 = ["/StorageCitlalli/DAS/DataCDMX/2022/299/SR_DS_2022-10-26_12-09-01_UTC.h5", (117-0.5,123),(100,1000),[19.368,-99.172],488]#33 # 488
fl35 = ["/StorageCitlalli/DAS/DataCDMX/2022/292/SR_DS_2022-10-19_03-48-03_UTC.h5", (191-0.25,197),(450,1400),[19.399,-99.162],650]#34  #650 


fl37 = ["/StorageCitlalli/DAS/DataCDMX/2022/206/SR_DS_2022-07-25_18-13-31_UTC.h5", (252,258),(100,1000),[19.366,-99.181],1180]#36 #1180  <<
fl39 = ["/StorageCitlalli/DAS/DataCDMX/2022/155/SR_DS_2022-06-04_04-07-53_UTC.h5", (294+1,302), (0,2000),[19.446098070773672,-99.105705004952980],4020]#37 ### 4020 metros a la fibra 
fl40 = ["/StorageCitlalli/DAS/DataCDMX/2022/151/SR_DS_2022-05-31_15-13-17_UTC.h5", (88,94),(400,1000),[19.392,-99.165],904]#38  904
fl41 = ["/StorageCitlalli/DAS/DataCDMX/2022/148/SR_DS_2022-05-28_16-30-55_UTC.h5", (476-0.25,482),(400,1500),[19.39,-99.166],980]#39  980
fl42 =  ["/StorageCitlalli/DAS/DataCDMX/2022/148/SR_DS_2022-05-28_09-20-52_UTC.h5", (588,596), (100,1100),[19.37,-99.182],1480]#40  1480
fl43 = ["/StorageCitlalli/DAS/DataCDMX/2022/146/SR_DS_2022-05-26_07-00-52_UTC.h5", (359-0.25,365),(100,1100),[19.255,-99.372],21000] #41  21000
fl44 = ["/StorageCitlalli/DAS/DataCDMX/2023/38/SR_DS_2023-02-07_14-48-00_UTC.h5", (348+1.2,354),(0,1100),[19.365,-99.184],1430] #42  1430   <<--

fl45 = ["/StorageCitlalli/DAS/DataCDMX/2023/154/SR_DS_2023-06-03_01-10-47_UTC.h5",(78-0.25,84),(100,1100),[19.371,-99.177],1100]#43   ## 1100 metros 
fl46 = ["/StorageCitlalli/DAS/DataCDMX/2023/153/SR_DS_2023-06-02_11-26-45_UTC.h5",(174-0.25,180),(100,1100)]#45 ##  ## no hay localizacion en .loc ni en Pick

fl47 = ["/StorageCitlalli/DAS/DataCDMX/2023/151/SR_DS_2023-05-31_11-56-40_UTC.h5",(228-0.25,234),(650,1500),[19.421,-99.138],1220]#46 ## 1220 metros
fl48 = ["/StorageCitlalli/DAS/DataCDMX/2023/149/SR_DS_2023-05-29_10-26-40_UTC.h5",(124-0.25,130),(600,1000),[19.401,-99.137],754]## 754 metros 
fl49 = ["/StorageCitlalli/DAS/DataCDMX/2023/142/SR_DS_2023-05-22_08-43-04_UTC.h5",(165-0.25,171),(0,700),[19.712,-98.741],52200]# 5 #### 52200 metros de distancia a la fibra
fl50 = ["/StorageCitlalli/DAS/DataCDMX/2023/142/SR_DS_2023-05-22_08-23-04_UTC.h5",(451-0.25,457),(0,700),[19.545,-98.824],34300]#3 ##### 34300 m (34.3 km) distancia a la fibra
fl51 = ["/StorageCitlalli/DAS/DataCDMX/2023/142/SR_DS_2023-05-22_08-13-04_UTC.h5",(424-0.25,430),(0,1000),[19.487,-98.979],17000] #2  #####17000m (17 km) distancia a la fibra 
fl52 = ["/StorageCitlalli/DAS/DataCDMX/2023/142/SR_DS_2023-05-22_05-03-04_UTC.h5",(150-0.25,156),(100,1000),[19.370877178960683,-99.178125335647550],1210 ] 
fl53 = ["/StorageCitlalli/DAS/DataCDMX/2023/137/SR_DS_2023-05-17_04-02-13_UTC.h5",(412-0.25,418),(100,1000),[19.37,-99.184],1660]## 1660 metros 
fl54 = ["/StorageCitlalli/DAS/DataCDMX/2023/136/SR_DS_2023-05-16_10-42-13_UTC.h5",(500-0.6,506),(100,1000),[19.366,-99.178],897]## 897 metros 
fl55 = ["/StorageCitlalli/DAS/DataCDMX/2023/135/SR_DS_2023-05-15_08-42-13_UTC.h5",(222-0.25,228),(0,1000),[19.367,-99.192],2250]## 2250 metros 
fl56 = ["/StorageCitlalli/DAS/DataCDMX/2023/135/SR_DS_2023-05-15_00-32-13_UTC.h5",(531-0.25,537),(0,1000),[19.36,-99.187],1340]## 1340 metros
fl57 = ["/StorageCitlalli/DAS/DataCDMX/2023/132/SR_DS_2023-05-12_23-42-38_UTC.h5",(254-0.25,260),(0,1000),[19.364,-99.18],998]## 998 metros
fl58 = ["/StorageCitlalli/DAS/DataCDMX/2023/132/SR_DS_2023-05-12_03-52-38_UTC.h5",(533-0.5,539),(0,1000),[19.366,-99.187],1780]### 1780 metros ### (19.366	-99.187)
fl59 = ["/StorageCitlalli/DAS/DataCDMX/2023/131/SR_DS_2023-05-11_16-52-38_UTC.h5",(191-0.25,197),(0,1000),[19.365,-99.178],849]####  849 ### (19.365	-99.178)
fl60 = ["/StorageCitlalli/DAS/DataCDMX/2023/131/SR_DS_2023-05-11_15-52-38_UTC.h5",(333,339),(50,1000),[19.367,-99.182],1320] #6--- 1320 metros a la fibra (19.367	-99.182) 
fl61 = ["/StorageCitlalli/DAS/DataCDMX/2023/131/SR_DS_2023-05-11_15-22-38_UTC.h5", (167+0.25,173),(0,1000),[19.366,-99.185],1570] #3--  1570metros a la fibra (19.366	-99.185)
fl62 = ["/StorageCitlalli/DAS/DataCDMX/2023/131/SR_DS_2023-05-11_15-02-38_UTC.h5",(159,165),(50,1000),[19.365,-99.178],850]## 1--850 metros a la fibra (19.365	-99.178)
fl63 = ["/StorageCitlalli/DAS/DataCDMX/2023/131/SR_DS_2023-05-11_14-42-38_UTC.h5",(65-0.5,71),(50,1000),[19.366,-99.183],1380]##5--- 1380 metros a la fibra (19.366	-99.183)
fl64 = ["/StorageCitlalli/DAS/DataCDMX/2023/131/SR_DS_2023-05-11_14-32-38_UTC.h5",(352,358),(50,1000),[19.368,-99.183],1470]#4-- 1470 metros a la fibra (19.368	-99.183) 
fl65 = ["/StorageCitlalli/DAS/DataCDMX/2023/131/SR_DS_2023-05-11_13-52-38_UTC.h5",(299,335),(150,1000),[19.368,-99.167],35]#6-- 35 metros a la fibra (19.368	-99.167)
fl66 = ["/StorageCitlalli/DAS/DataCDMX/2023/131/SR_DS_2023-05-11_13-22-38_UTC.h5",(155-0.6,161),(0,1000),[19.359,-99.172],9]#3--- 9 metros de la fibra (19.359	-99.172)
fl67 = ["/StorageCitlalli/DAS/DataCDMX/2023/131/SR_DS_2023-05-11_07-42-38_UTC.h5",(47-0.75,53),(50,1000),[19.37,-99.186],1850] ## 1850 metros a la fibra 
fl68 = ["/StorageCitlalli/DAS/DataCDMX/2023/131/SR_DS_2023-05-11_04-32-38_UTC.h5",(137-0.5,143),(50,1000),[19.371130055166887,-99.189522765432570],2230 ]  ## 2230 metros a la fibra 
fl68_2 = ["/StorageCitlalli/DAS/DataCDMX/2023/131/SR_DS_2023-05-11_04-32-38_UTC.h5",(477-0.75,483),(0,1000), [19.363184865248062, -99.195016492787060], 2180 ] #2180 metros a la fibra
fl69 = ["/StorageCitlalli/DAS/DataCDMX/2023/131/SR_DS_2023-05-11_04-22-38_UTC.h5", (202+2,208),(0,1000),[19.364016359088264,-99.177275788150060],737] #3 ## 737 metros a la fibra 
fl70 = ["/StorageCitlalli/DAS/DataCDMX/2023/131/SR_DS_2023-05-11_04-12-38_UTC.h5",(461,467),(0,1000),[19.367270670249365,-99.190179901398800],2120] # #2120 metros a la fibra 
fl75 = ['/StorageCitlalli/DAS/DataCDMX/2023/53/SR_DS_2023-02-22_00-18-41_UTC.h5',(255-0.25,261),(0,1000),[19.359110877786797,-99.175365106720800], 339] ## 339 metros a la fibra
fl33 = ["/StorageCitlalli/DAS/DataCDMX/2022/322/SR_DS_2022-11-18_23-43-22_UTC.h5", (57,63),(1266,2266), [19.510886371061563,-99.137333916429610], 53 ] ## por allá en atzcapoltz
fl38 = ["/StorageCitlalli/DAS/DataCDMX/2022/198/SR_DS_2022-07-17_22-18-28_UTC.h5", (198,203),(0,1000),[19.322798859355316,-99.177562268662040], 73] # 73 metros de la fibra  # segun hoja de excel el hipo es [19.283,-99.066] a 7.560 km de segunda fibra y 12 km de primera fibra  << # 



In [5]:

## convertir a herencia de Model
class modeloFH(tf.keras.Model):
    def __init__(self):
        super(modeloFH, self).__init__()
        #creating layers in initializer
        self.conv1 = Conv2D(filters=16, kernel_size=(3,3), padding="same", activation="relu")

        self.drop1 = Dropout(0.2)

        
        
        self.conv2 = Conv2D(filters=16, kernel_size=(3,3), padding="same")#, activation="relu")
        self.bt1 = BatchNormalization()#activation="relu")
        self.rl1 = Activation('relu')
        self.drop2 = Dropout(0.2)
        self.conv3 = Conv2D(filters=16, kernel_size=(3,3), strides=(2, 2), padding="valid")

        
        self.conv4 = Conv2D(filters=32, kernel_size=(3,3), padding="same")#, activation="relu")
        self.bt2 = BatchNormalization()
        self.rl2 = Activation('relu')
        self.drop3 = Dropout(0.2)
        self.conv5 = Conv2D(filters=32, kernel_size=(3,3), strides=(2, 2), padding="valid")


        self.conv6 = Conv2D(filters=32, kernel_size=(3,3), padding="same")#, activation="relu")
        self.bt3 = BatchNormalization()
        self.rl3 = Activation('relu')
        self.drop4 = Dropout(0.2)
        self.conv7 = Conv2D(filters=32, kernel_size=(3,3), strides=(2, 2), padding="valid")

        self.conv8 = Conv2D(filters=64, kernel_size=(3,3), padding="same")#, activation="relu")
        self.bt4 = BatchNormalization()
        self.rl4 = Activation('relu')
        self.drop5 = Dropout(0.2)
        self.conv9 = Conv2D(filters=64, kernel_size=(3,3), strides=(2, 2), padding="valid")

        
        
  
        self.flatten = Flatten()
        self.fc1 = Dense(units=64)#, activation="relu")
        self.drop6 = Dropout(0.4)
        self.rl5 = Activation('relu')

        self.fc2 = Dense(units = 1, activation="sigmoid")#(units = 1, activation="sigmoid")


    def call(self, input_tensor):
        x = self.conv1(input_tensor)
        x = self.drop1(x)
        x = self.conv2(x)
        x = self.bt1(x)
        x = self.rl1(x)
        x = self.drop2(x)
        x = self.conv3(x)
        
        x = self.conv4(x)
        x = self.bt2(x)
        x = self.rl2(x)
        x = self.drop3(x)
        x = self.conv5(x)
        x = self.conv6(x)
        x = self.bt3(x)
        x = self.rl3(x)
        x = self.drop4(x)
        x = self.conv7(x)
        x = self.conv8(x)
        x = self.bt4(x)
        x = self.rl4(x)
        x = self.drop5(x)
        x = self.conv9(x)
       
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.drop6(x)
        x = self.rl5(x)
        x = self.fc2(x)
        return x

input_layer = Input(shape=(1000, 120, 1,))
x = modeloFH()(input_layer)
model = Model(inputs=input_layer, outputs=x)





2025-01-23 18:13:51.737378: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/gridengine/lib/lx-amd64:/opt/openmpi/lib
2025-01-23 18:13:51.737422: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2025-01-23 18:13:51.737448: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (compute-0-4.local): /proc/driver/nvidia/version does not exist
2025-01-23 18:13:51.737949: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
######## ######## ######### #########
### #### #### #### #### #### #### ###

###  Todos los conjuntos empleados hasta el momento, que han producido los mejores resultados en evaluaciones independientes
##
# Horizontales  
# Vertical 
# Ruido  
# time shift


######## ######## ######### #########
### #### #### #### #### #### #### ###
####################### 13 de marzo 2024, 10 abril 2024
##



hf = h5py.File('/home/aortiza/CDMX/DataAugmentation/X_event_H.h5', 'r') 

X_event_H =  hf.get('dataset_1')

X_event_H = np.array(X_event_H)

hf.close()

X_event_HCp = X_event_H.copy()

### normalizar usando min max por renglón en X_events, que significa considerando todo el parche de cada 6segs. 
##de cada ejemplo

for i,signal in enumerate(X_event_H):
    MEAN = signal.mean()
    STD = signal.std()
    
    X_event_HCp[i] = (signal-MEAN)/(STD)
   
del(X_event_H)






In [7]:
X_event_HCp = X_event_HCp[-3000:,:]

In [35]:

hf = h5py.File('/home/aortiza/CDMX/DataAugmentation/X_event_V.h5', 'r') 

X_event_V =  hf.get('dataset_1')

X_event_V = np.array(X_event_V)

hf.close()

X_event_VCp = X_event_V.copy()

### normalizar usando min max por renglón en X_events, que significa considerando todo el parche de cada 6segs. 
##de cada ejemplo

for i,signal in enumerate(X_event_V):
    MEAN = signal.mean()
    STD = signal.std()
    
    X_event_VCp[i] = (signal-MEAN)/(STD)
   
del(X_event_V)

In [9]:
X_event_VCp = X_event_VCp[-3000:,:]

In [10]:
X_event_HCp.shape

(3000, 120000)

In [11]:
X_event_VCp.shape

(3000, 120000)

In [12]:
X_signal1 = np.append(X_event_HCp,X_event_VCp,axis=0)


In [13]:
X_signal1.shape

(6000, 120000)

In [14]:
XtrainSets = ["/X_NoiseCummulative10.h5","/X_NoiseCummulative5.h5","/X_NoiseCummulative4.h5","/X_snr3_5orMore.h5","/X_NoiseCummulative3.h5","/X_snr2_5orMore.h5","/X_NoiseCummulative2.h5"
,"/X_snr1_5orMore.h5","/X_NoiseCummulative1.h5","/X_snr085orMore.h5","/X_snr075orMore.h5"
,"/X_snr06orMore.h5","/X_NoiseCummulative05.h5", "/X_snr04orMore.h5","/X_snr03orMore.h5","/X_snr02orMore.h5","/X_NoiseCummulative01.h5"]
 
    
#XtrainSets = ["/X_snr06orMore.h5"]

In [15]:
len(XtrainSets[:])

17

In [16]:
XtrainSets[12:]

['/X_NoiseCummulative05.h5',
 '/X_snr04orMore.h5',
 '/X_snr03orMore.h5',
 '/X_snr02orMore.h5',
 '/X_NoiseCummulative01.h5']

In [17]:
### Ruido
#### Cargar conjunto de entrenamiento con ruido
#X_flashH60.h5

hf = h5py.File('/home/aortiza/CDMX/DataAugmentation/X_flashH60.h5', 'r')

X_flash = hf.get('dataset_1')

X_flash = np.array(X_flash)

hf.close()
X_flashCp = X_flash.copy()

for i,evento in enumerate(X_flash):
    MEAN = evento.mean()
    STD = evento.std()
    
    X_flashCp[i] = (evento-MEAN)/(STD)
del(X_flash)  





In [18]:
hf = h5py.File('/home/aortiza/CDMX/DataAugmentation/X_microV50Metro12Aug.h5', 'r')

X_metroAug = hf.get('dataset_1')

X_metroAug = np.array(X_metroAug)

hf.close()
X_metroAugCp = X_metroAug.copy()

for i,evento in enumerate(X_metroAug):
    MEAN = evento.mean()
    STD = evento.std()
    
    X_metroAugCp[i] = (evento-MEAN)/(STD)
del(X_metroAug)  

In [19]:
hf = h5py.File('X_data_Noise_Test.h5', 'r')

X_noMicroTest =  hf.get('dataset_1')

X_noMicroTest = np.array(X_noMicroTest)

hf.close()

In [20]:
### conjunto para test de snr  



hf = h5py.File('/home/aortiza/CDMX/PrepararConjuntos/X_test_snr.h5', 'r') 

X_NoiseTest =  hf.get('dataset_1')

X_NoiseTest = np.array(X_NoiseTest)

hf.close()

X_NoiseTestCp = X_NoiseTest.copy()

### normalizar usando min max por renglón en X_events, que significa considerando todo el parche de cada 6segs. 
##de cada ejemplo

for i,signal in enumerate(X_NoiseTest):
    MEAN = signal.mean()
    STD = signal.std()
    
    X_NoiseTestCp[i] = (signal-MEAN)/(STD)
    
del(X_NoiseTest)





In [21]:

valSet = np.zeros((7,2),dtype=float)## primer índice para TP segundo indice para TN


In [22]:


hf = h5py.File('/home/aortiza/CDMX/Repositorio de ruido/X_data_Noise.h5', 'r')

X_ruido =  hf.get('dataset_1')

X_ruido = np.array(X_ruido)

hf.close()

X_ruidoCp = X_ruido.copy()
### normalizar usando min max por renglón en X_events, que significa considerando todo el parche de cada 6segs. 
##de cada ejemplo

for i,ruido in enumerate(X_ruido):
    MEAN = ruido.mean()
    STD = ruido.std()
    
    X_ruidoCp[i] = (ruido-MEAN)/(STD)
del(X_ruido)   

## ruido de Metro

hf = h5py.File('/home/aortiza/CDMX/Repositorio de ruido/X_data_Noise_Metro.h5', 'r')

X_ruidoMetro =  hf.get('dataset_1')

X_ruidoMetro = np.array(X_ruidoMetro)

hf.close()
X_ruidoMetroCp = X_ruidoMetro.copy()

for i,ruidillo in enumerate(X_ruidoMetro):
    MEAN = ruido.mean()
    STD = ruido.std()
    
    X_ruidoMetroCp[i] = (ruido-MEAN)/(STD)
   
del(X_ruidoMetro)

In [23]:

X_ruidoTrain = np.append(X_ruidoCp,X_ruidoMetroCp,axis=0)

X_ruidoTrain = np.append(X_ruidoTrain,X_metroAugCp,axis=0)
X_ruidoTrain = np.append(X_ruidoTrain,X_flashCp,axis=0)
del(X_ruidoCp,X_metroAugCp,X_ruidoMetroCp,X_flashCp)



In [24]:
X_ruidoTrain.shape

(8720, 120000)

In [25]:
### Etiquetas
X_label_1_all = np.ones((9000),dtype=int)

#X_label_0 = np.zeros((X_ruidoCp.shape[0])+(X_ruidoMetroCp.shape[0])+(X_flashCp.shape[0]), dtype=int )  ### Labels deben ser enteras int
X_label_0 = np.zeros((8720), dtype=int ) 


In [ ]:
## lista de conjuntos sobre los que vamos a iterar:



data_dir = pathlib.Path("/home/aortiza/CDMX/PrepararConjuntos/")
for j,file in enumerate(XtrainSets[:6]):
    print(str(data_dir)+file)
    
    h5 = h5py.File(str(data_dir)+file,'r')

    data =  h5.get('dataset_1')

    data = np.array(data)

    h5.close()

    X_signal = np.append(X_signal1,data.reshape(3000,120000),axis=0)
    
    input_layer = Input(shape=(1000, 120, 1,))
    x = modeloFH()(input_layer)
    model = Model(inputs=input_layer, outputs=x)



    callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10)

    model.compile(optimizer=Adam(learning_rate=1e-4),loss='binary_crossentropy',#tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy']) ## usar metrica precision. Pero mejor Matthews  



#history = model.fit(np.append(compM( X_event_V30Cp,6), compM(X_ruidoCp[:1100],6),axis=0).reshape(2200,1000,120), np.append(len(X_label_1_all[:1100])*[(1,0)], len(X_label_0[:1100])*[(0,1)],axis=0), #(X_label_1_all[:]), (X_label_0[:]),axis=0
                    


#history = model.fit(np.append(compM(np.append(X_event_V90Cp,X_event_V100Cp,axis=0),6), compM(np.append(X_ruidoCp[:],X_metroAugCp,axis=0),6),axis=0).reshape(5200,1000,120), np.append(len(X_label_1_all[:2200])*[(1,0)], (3000)*[(0,1)],axis=0),


#history = model.fit(np.append(compM(np.append(X_event_V70Cp,X_event_V80Cp,axis=0),4), compM(np.append(X_ruidoCp[:],X_metroAugCp,axis=0),4),axis=0).reshape(5200,1000,120), np.append(X_label_1_all[:2200], X_label_0,axis=0),
                       
#history = model.fit(np.append(stdTrace(np.append(X_event_V130Cp,X_event_V120Cp,axis=0)), stdTrace(np.append(X_ruidoCp[:],X_metroAugCp,axis=0)),axis=0).reshape(5200,1000,120), np.append(X_label_1_all[:2200], X_label_0,axis=0),
                     
    history = model.fit(np.append(compM(X_signal.reshape(9000,120000),4), compM(X_ruidoTrain,4),axis=0).reshape(9000+8720,1000,120), np.append(X_label_1_all, X_label_0,axis=0),
                     
                   
#history = model.fit(np.append( (X_NoiseCp,2).reshape(3000,1000*120), compM(X_ruidoTrain,2),axis=0).reshape(7186,1000,120), np.append(len(X_label_1_all[:3000])*[(1,0)], len(X_label_0[:4186])*[(0,1)],axis=0),

#history = model.fit(np.append( (X_NoiseCp[900*13:900*14]).reshape(900,1000*120), X_ruidoTrain,axis=0).reshape(5086,1000,120), np.append(len(X_label_1_all[:900])*[(1,0)], len(X_label_0[:4186])*[(0,1)],axis=0),

            #batch_size=8,
              #epochs=nb_epoch,
              callbacks=[callback],
              validation_split=0.05,
              epochs=40,
              shuffle=True,
              verbose=1)
        ##### Evaluar modelo en 
            
                #3600-360+i*10+8
    microTest = model.predict(compM((X_NoiseTestCp).reshape(18,1000,120),4))
    tPos = 0
    fNeg = 0
        
        
    for i in microTest:
            #if i.argmax() == 0:
        if i[0] >= 0.999:
            tPos +=1
        else:
            fNeg+=1
    print("tPos = "+str(tPos)+', '+"fNeg = "+str(fNeg))
        
            #print(i.argmax())
        
        
            
        
    #noMicroTest = model.predict(stdTrace(X_noMicroTest.reshape(1000,1000,120)))   
    noMicroTest = model.predict(compM(X_noMicroTest.reshape(1000,1000,120),4)) 
        #noMicroTest = model.predict(OB(X_noMicroTest[0:1000].reshape(1000,1000,120))) 
        #noMicroTest = model.predict(maxGlob(X_nomicroTest).reshape(1000,1000,120)) 
        #noMicroTest = model.predict((X_nomicroTest).reshape(1000,1000,120)) 
    tNeg = 0
    fPos = 0
        
    for i in noMicroTest:
            #if i.argmax() == 1:
        if i[0] <= 0.999:
            tNeg +=1
        else:
            fPos+=1
    print("tNeg = "+str(tNeg)+', '+"fPos = "+str(fPos))
        
    valSet[j,0] = tPos

    valSet[j,1] = tNeg  
    
    
    
    del(model)


/home/aortiza/CDMX/PrepararConjuntos/X_NoiseCummulative10.h5
Epoch 1/40
527/527 [==============================] - 386s 730ms/step - loss: 0.4685 - accuracy: 0.7374 - val_loss: 0.0014 - val_accuracy: 1.0000
Epoch 2/40
527/527 [==============================] - 396s 752ms/step - loss: 0.1038 - accuracy: 0.9606 - val_loss: 0.0149 - val_accuracy: 0.9955
Epoch 3/40
527/527 [==============================] - 386s 733ms/step - loss: 0.0558 - accuracy: 0.9794 - val_loss: 0.0743 - val_accuracy: 0.9661
Epoch 4/40
527/527 [==============================] - 398s 755ms/step - loss: 0.0318 - accuracy: 0.9894 - val_loss: 0.0621 - val_accuracy: 0.9707
Epoch 5/40
527/527 [==============================] - 387s 735ms/step - loss: 0.0264 - accuracy: 0.9904 - val_loss: 0.3768 - val_accuracy: 0.9063
Epoch 6/40
527/527 [==============================] - 384s 729ms/step - loss: 0.0215 - accuracy: 0.9929 - val_loss: 0.0682 - val_accuracy: 0.9740
Epoch 7/40
527/527 [==============================] - 387s 734m

In [27]:
valSet# [12:17]

array([[ 17., 995.],
       [ 17., 954.],
       [ 15., 914.],
       [ 16., 990.],
       [ 17., 975.],
       [  0.,   0.],
       [  0.,   0.]])

In [ ]:
[6:12]

array([[  13.,  765.],
       [  14.,  831.],
       [  18.,  960.],
       [  15.,  983.],
       [  16., 1000.],
       [  17.,  975.]])

In [29]:
[6:12]

valSet

array([[ 15., 994.],
       [ 15., 916.],
       [ 18., 937.],
       [ 15., 960.],
       [ 15., 985.],
       [  0.,   0.],
       [  0.,   0.]])

In [30]:
XtrainSets[6:12]

['/X_NoiseCummulative2.h5',
 '/X_snr1_5orMore.h5',
 '/X_NoiseCummulative1.h5',
 '/X_snr085orMore.h5',
 '/X_snr075orMore.h5',
 '/X_snr06orMore.h5']

In [34]:
XtrainSets[11]

'/X_snr06orMore.h5'

In [32]:

valSet

array([[ 15., 994.],
       [ 15., 916.],
       [ 18., 937.],
       [ 15., 960.],
       [ 15., 985.],
       [  0.,   0.],
       [  0.,   0.]])

In [26]:
valSet

array([[ 18., 940.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.]])

In [32]:
valSet

array([[ 18., 926.],
       [ 17., 857.],
       [ 18., 836.],
       [ 15., 968.],
       [ 16., 692.],
       [ 17., 953.],
       [  0.,   0.]])

In [31]:
valSet

array([[  18.,  989.],
       [  15., 1000.],
       [  16., 1000.],
       [  15.,  998.],
       [  17.,  976.],
       [   0.,    0.],
       [   0.,    0.]])

In [34]:
XtrainSets[12:]

['/X_NoiseCummulative05.h5',
 '/X_snr04orMore.h5',
 '/X_snr03orMore.h5',
 '/X_snr02orMore.h5',
 '/X_NoiseCummulative01.h5']

In [37]:
valSet #[12:]

array([[ 18., 515.],
       [ 14., 991.],
       [ 15., 964.],
       [ 14., 944.],
       [ 14., 955.],
       [ 16., 997.],
       [  0.,   0.]])